### Data preparation

Importing the libraries we will use to solve the linear programming challenge

In [1]:
import numpy as np
import math
import pandas as pd
import setup
from docplex.mp.model import Model
from docplex.mp.conflict_refiner import ConflictRefiner


Reading the xslx files with all the data of the problem

In [2]:
dfdemand = pd.read_excel('data_or_test.xlsx', sheet_name='Demand')
dfMarkup = pd.read_excel('data_or_test.xlsx', sheet_name='Markup')
dfLastmile = pd.read_excel('data_or_test.xlsx', sheet_name='Last Mile')
dfStock = pd.read_excel('data_or_test.xlsx', sheet_name='Stock')
dfIniStock = pd.read_excel('data_or_test.xlsx', sheet_name='Initial Stock')
dfTransfers = pd.read_excel('data_or_test.xlsx', sheet_name='Transfers')
dfInbound = pd.read_excel('data_or_test.xlsx', sheet_name='Inbound')
dfProduct = pd.read_excel('data_or_test.xlsx', sheet_name='Product')

Visualizing the data as a dataframe and checking if it's in the correct format 

In [3]:
dfdemand.head()

,State,Seller,Product,Week,Demand (unit),Price
0,ST1,S1,P1,W1,668,1449.19
1,ST1,S1,P1,W2,887,1983.44
2,ST1,S1,P1,W3,499,1386.48
3,ST1,S1,P1,W4,281,889.35
4,ST1,S2,P2,W1,144,2115.55


Defining the list of values in which our variables will span. 

In [4]:
list((dfTransfers["Category"]).unique())

['C1', 'C2', 'C3', 'C4', 'C5']

In [5]:
days = list(range(1,29 )) # DEBERIA SER 29 CAMBIAR!!!!!!!!!!!!!!!!!!!!!!!!!!!
weeks =  list((dfdemand["Week"]).unique())
fcenters = list((dfTransfers["FC"]).unique())
sellers = list((dfdemand["Seller"]).unique())
states = list((dfdemand["State"]).unique())
products = list((dfdemand["Product"]).unique())
categories = list((dfTransfers["Category"]).unique())

Example:

In [6]:
print(states)

['ST1', 'ST2', 'ST3', 'ST4', 'ST5', 'ST6', 'ST7', 'ST8', 'ST9', 'ST10']


# The mathematical model

### Variables

We define the CPLEX model we will be using

In [7]:
mdl = Model(name="modelEcoMole")

 When creating variables, we will use the following type format:
 B,I,C,N,S or type short names (e.g.: binary, integer, continuous, semicontinuous, semiinteger)

stockAtFC represents the amount of stock that a given fullfilment center has at a given day of a given product. Therefore it must be any non negative integer.

In [8]:
stockAtFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(stockAtFC)

25200

transferBetweenFCs represents the amount of products that a given fullfilment center transfers to another fullfilment center at a given day of a given product. Therefore it must be any non negative integer.

In [9]:
transferBetweenFCs = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, fcenters, days, products])
len(transferBetweenFCs)

75600

deliveryToStateFromFC represents the amount of product that a given fullfilment center transfers to a client at a given state at a given day. 

In [10]:
deliveryToStateFromFC = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, states, days, products])
len(deliveryToStateFromFC)

252000

amountInboundAtFCProd represents the amount of product that a given fullfilment center recives from a seller at a given day.

In [11]:
amountInboundAtFCProd = mdl.var_hypercube(vartype_spec='I', seq_of_keys=[fcenters, days, products])
len(amountInboundAtFCProd)

25200

### Auxiliar functions

In [86]:
def giveCategoryOfProduct(p):
    return(list(dfProduct[dfProduct["Product"] == p]["Category"])[0])

In [87]:
giveCategoryOfProduct("P2")

'C5'

In [14]:
def giveSellerOfProduct(p):
    return(list(dfIniStock[dfIniStock["Product"] == p]["Seller"])[0])

In [15]:
giveSellerOfProduct("P124")

'S24'

In [16]:
def giveDemandOfProduct(p, sta, w):
    return(list(dfdemand[(dfdemand["Product"] == p)&(dfdemand["State"] == sta)&(dfdemand["Week"] == w)]["Demand (unit)"])[0])

In [17]:
giveDemandOfProduct("P3", "ST1", "W3")

397

In [18]:
def givePriceOfProduct(p, sta, w):
    return(list(dfdemand[(dfdemand["Product"] == p)&(dfdemand["State"] == sta)&(dfdemand["Week"] == w)]["Price"])[0])

In [19]:
givePriceOfProduct("P2","ST1", "W4")

1312.53

In [20]:
def giveLastMileCostOfProduct(p, sta, fc):
    c = giveCategoryOfProduct(p)
    return(list(dfLastmile[(dfLastmile["Category"] == c)&(dfLastmile["State"] == sta)&(dfLastmile["FC"] == fc)]["Cost (per unit)"])[0])

In [21]:
giveLastMileCostOfProduct("P2", "ST4", "FC3")

34.0

In [22]:
def giveWeekOfDay(d):
    w = math.ceil((d/28 * 4))
    weekAsString = "W"+str(w)
    return(weekAsString)

In [23]:
giveWeekOfDay(11)

'W2'

In [24]:
def giveSafetyStock(p, fc):
    return(list(dfStock[(dfStock["Product"] == p) & (dfStock["FC"] == fc)]["Safety Stock (days of sales)"])[0])

In [25]:
giveSafetyStock("P4","FC2")

5

In [26]:
def giveCostOfInbound(s,c,fc):
    return(list(dfInbound[(dfInbound["Seller"] == s) & (dfInbound["Category"] == c) & (dfInbound["FC"] == fc) ]["Cost (per unit)"])[0])

In [27]:
giveCostOfInbound("S2","C3","FC2")

30.25

In [28]:
def giveTimeOfInbound(s,c,fc):
    return(list(dfInbound[(dfInbound["Seller"] == s) & (dfInbound["Category"] == c) & (dfInbound["FC"] == fc) ]["Time (days)"])[0])

In [29]:
giveTimeOfInbound("S2","C4","FC2")

3

In [30]:
def giveCostOfStorage(p, fc):
    return(list(dfStock[(dfStock["Product"] == p)&(dfStock["FC"] == fc)]["Cost (per day and unit)"])[0])

In [31]:
giveCostOfStorage("P5", "FC2")

5.36

In [32]:
def giveCostOfTransfer(fc1, fc2, c):
    return(list(dfTransfers[(dfTransfers["FC"] == fc1)&(dfTransfers["FC.1"] == fc2) &(dfTransfers["Category"] == c) ]["Cost (per unit)"])[0])

In [33]:
giveCostOfTransfer("FC3", "FC1", "C4")

10.37

In [34]:
def giveTimeOfTransfer(fc1, fc2, c):
    return(list(dfTransfers[(dfTransfers["FC"] == fc1)&(dfTransfers["FC.1"] == fc2) &(dfTransfers["Category"] == c) ]["Time (days)"])[0])

In [35]:
giveTimeOfTransfer("FC3", "FC1", "C4")

3

In [36]:
def giveMarkUpOfSeller(s):
    return(list(dfMarkup[(dfMarkup["Seller"] == s)]["Markup (%)"])[0])

In [37]:
giveMarkUpOfSeller("S4")

7

In [38]:
def giveInitialStock(p, fc):
    return(list(dfIniStock[(dfIniStock["Product"] == p)&(dfIniStock["FC"] == fc)]["Quantity (unit)"])[0])

In [39]:
giveInitialStock("P190", "FC3")

162

In [40]:
def giveAverageDemandPerDay(p): #Returns daily average demand  (all states and all weeks).
    return(sum(list(dfdemand[(dfdemand["Product"] == p)]["Demand (unit)"]))/(len(states)*len(weeks)*len(fcenters)))
    

In [41]:
giveAverageDemandPerDay("P123")

179.66666666666666

### Objective function and related restrictions

Inbound costs are dependant on the amount of product moving and which fullfilment center will be de the destination.

In [42]:
InboundCosts = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for p in products:
            c = giveCategoryOfProduct(p)
            s = giveSellerOfProduct(p)
            costThisCSFC = giveCostOfInbound(s,c,fc)
            InboundCosts = InboundCosts + costThisCSFC * amountInboundAtFCProd[fc,d,p]

Storage costs are dependant on the amount of product at a particular fulfilment center and which fullfilment center will be de the destination.

In [43]:
StorageCost = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for p in products:
            costThisP = giveCostOfStorage(p, fc)
            StorageCost = StorageCost + costThisP * stockAtFC[fc,d,p]

In [44]:
TransfersCosts = 0
for d in days:
    #print(d)
    for fc1 in fcenters:
        for fc2 in fcenters:
            if(fc1 != fc2):
                for p in products:
                    c = giveCategoryOfProduct(p)
                    costThisTran = giveCostOfTransfer(fc1, fc2, c)
                    TransfersCosts = TransfersCosts + costThisTran * transferBetweenFCs[fc1,fc2,d,p]

Los costos de last mile no usan demanda porque la restriccion de la demanda esta en otro lado

In [45]:
LastMileCosts = 0
for d in days:
    #print(d)
    for fc in fcenters:
        for st in states:
            for p in products:
                costThisLastmile = giveLastMileCostOfProduct(p, st, fc)
                LastMileCosts = LastMileCosts + costThisLastmile * deliveryToStateFromFC[fc,st,d,p]

In [46]:
TotalCosts = InboundCosts + StorageCost + TransfersCosts + LastMileCosts

Note that revenue from markup is not a variable we can change. ()

Therefore defining: Profit = Revenue - Costs 

is only dependant on Costs. That means that the solutions to maximize(Profit) are equivalent to the ones that minimize(Costs)

In [47]:
revenue = 0
for w in weeks:
    for p in products:
        for st in states:
            demandaThis = giveDemandOfProduct(p, st, w)
            priceThis = givePriceOfProduct(p, st, w)
            s = giveSellerOfProduct(p)
            markupThis = giveMarkUpOfSeller(s)
            revenue = revenue + (markupThis/100) * priceThis * demandaThis

In [48]:
print(revenue)

1369020027.1043003


In [49]:
Profit = revenue - TotalCosts

In [50]:
mdl.maximize(Profit)

### Stock restrictions

We set the initial stock

In [51]:
for fc in fcenters:
    for p in products:
        mdl.add_constraint_(stockAtFC[fc, 1, p] == giveInitialStock(p, fc)) #On day 1 we have the initial stock

The stock at a given day is the stock the stock of yesterday + stock arrived - the stock that departed.
The stock can arrive or departure either from sellers or other fullfilment centers.

In [52]:
for d in days:
    if(d!=1):
        for fc1 in fcenters:
            for p in products:
                c = giveCategoryOfProduct(p)
                s = giveSellerOfProduct(p)
                arrivals = 0
                departures = 0
                for fc2 in fcenters:
                    if(fc1!=fc2):
                        potentialTimeArrival = giveTimeOfTransfer(fc2, fc1, c)
                        dprev = d - potentialTimeArrival
                        if(dprev>=1):
                            arrivals = arrivals + transferBetweenFCs[fc2,fc1,dprev,p]
                        departures = departures + transferBetweenFCs[fc1,fc2,d,p] 
                
                potentialTimeInbound = giveTimeOfInbound(s,c,fc1)
                dprev2 = d - potentialTimeInbound
                if(dprev2>=1):
                    arrivals = arrivals + amountInboundAtFCProd[fc1,d,p]
                for st in states:
                    departures = departures + deliveryToStateFromFC[fc1, st, d, p]
                mdl.add_constraint_(stockAtFC[fc1, d, p] == stockAtFC[fc1, d-1, p] + arrivals - departures)

Note that as we dont count as in store while on transfer between fulffilment centers (no storage cost while in transit) it is possible to sometimes be cheaper to make a round trip beetwen fulffilment centers than to keep them in place

Setting the minimum stock that must be in storage. We assume daily demand as an average across all weeks and across all states, divided by all fulfillment centers. so its

(sum of all demands from all states and all weeks ) divided by 
(number of weeks * number of states * number of fulfillment centers )

Also the restriction only holds when there was already a chance to take an inbound shipment, if we are still on the first few days when we have limited stock and we dont yet have safety stock the solution is still feasible.

In [53]:
for fc in fcenters:
    for p in products:
        s = giveSellerOfProduct(p)
        c = giveCategoryOfProduct(p)
        for d in days:
            tempArrival = giveTimeOfInbound(s,c,fc)
            if(d>tempArrival & tempArrival!=0):
                minDays = giveSafetyStock(p,fc)
                demThis = giveAverageDemandPerDay(p)
                mdl.add_constraint_(minDays*demThis <= stockAtFC[fc, d, p])

In [54]:
fc = "FC2"
p = "P136"
minDays = giveSafetyStock(p,fc)
demThis = giveAverageDemandPerDay(p)
print(minDays*demThis)

1438.6666666666667


### Transfers restrictions

Obviously there is no auto-transfering to the same fullfilment center 

In [55]:
for fc in fcenters:
    for d in days:
        for p in products:
            mdl.add_constraint_(transferBetweenFCs[fc,fc,d,p]==0)

Can't transfer more than what you currently have

In [56]:
for fc1 in fcenters:
    for fc2 in fcenters:
        for d in days:
            for p in products:
                mdl.add_constraint_(transferBetweenFCs[fc1,fc2,d,p]<=stockAtFC[fc1, d, p])

### Demand restrictions

In [57]:
giveDemandOfProduct("P2","ST3","W4")

149

In [58]:
for st in states:
    for p in products:
        for fc in fcenters:
            for w in weeks:
                delivered = 0
                for d in days:
                    w2 = giveWeekOfDay(d)
                    if(w2==w):
                        delivered = delivered + deliveryToStateFromFC[fc,st,d,p]
                dem = giveDemandOfProduct(p,st,w)
                mdl.add_constraint_(dem==delivered)

Can't transfer more than what you currently have

### Solution

In [59]:
msol = mdl.solve()

In [63]:
msol.get_value(Profit)

98913072.62430103

In [68]:
msol.get_value(TotalCosts)

1270106954.480012

In [69]:
# Costo permitiendo autoTransferencias
#718288.9800000015


In [70]:
msol.get_value(TransfersCosts)

36098666.680000916

In [71]:
msol.get_value(StorageCost)

60681515.92000856

In [73]:
msol.get_value(InboundCosts)

447579896.57999

In [74]:
msol.get_value(LastMileCosts)

725746875.3000017

In [ ]:
#a)

In [85]:
for d in days:
    print()
    print()
    print()
    print("Day " + str(d) +" transfers are: ")
    for p in products:
        for fc1 in fcenters:
            for fc2 in fcenters:
                val = msol.get_value(transferBetweenFCs[fc1,fc2,d,p])
                if(val>0.5):
                    print("Transfered "+ str(int(round(val))) +" of product "+str(p)+ " from  " + str(fc1) + " to  " + str(fc2))
            




Day 1 transfers are: 
Transfered 101 of product P1 from  FC1 to  FC3
Transfered 180 of product P1 from  FC2 to  FC1
Transfered 180 of product P1 from  FC2 to  FC3
Transfered 54 of product P1 from  FC3 to  FC1
Transfered 54 of product P1 from  FC3 to  FC2
Transfered 114 of product P2 from  FC1 to  FC3
Transfered 51 of product P2 from  FC2 to  FC3
Transfered 24 of product P2 from  FC3 to  FC1
Transfered 13 of product P3 from  FC1 to  FC2
Transfered 13 of product P3 from  FC1 to  FC3
Transfered 86 of product P3 from  FC2 to  FC3
Transfered 76 of product P3 from  FC3 to  FC1
Transfered 76 of product P3 from  FC3 to  FC2
Transfered 28 of product P4 from  FC1 to  FC3
Transfered 73 of product P4 from  FC2 to  FC1
Transfered 73 of product P4 from  FC2 to  FC3
Transfered 15 of product P4 from  FC3 to  FC1
Transfered 15 of product P4 from  FC3 to  FC2
Transfered 132 of product P5 from  FC1 to  FC3
Transfered 45 of product P5 from  FC2 to  FC1
Transfered 45 of product P5 from  FC2 to  FC3
Tran